# Awards

In [1]:
import time

In [2]:
start_time = time.time()

## 1. Define a time period for WC_CRM_1_ / WC_CRM_7_ and A1_CRM_

In [3]:
start_date_1 = '2023-10-01'
end_date_1 = '2023-10-31'

## 2. Define a time period for Raffle_laptops_

10th number of observed period + 1st period of the next month !!

In [4]:
start_date_2 = '2023-10-10'
end_date_2 = '2023-11-01'

## 3. Add Input Link

In [5]:
WC_CRM_1__path = "C:/Users/aleksandar.dimitrov/Desktop/PD/Input/Awards_templates/WC_CRM_1_October_2023_BAR.xlsx"

In [6]:
WC_CRM_7__path = "C:/Users/aleksandar.dimitrov/Desktop/PD/Input/Awards_templates/WC_CRM_7_October_2023_BAR.xlsx"

In [7]:
A1_CRM__path= "C:/Users/aleksandar.dimitrov/Desktop/PD/Input/Awards_templates/A1_CRM_October_2023_BAR.xlsx"

In [8]:
Raffle_laptops__path = "C:/Users/aleksandar.dimitrov/Desktop/PD/Input/Awards_templates/Raffle_Philips_10.2023_BAR.xlsx"

## 3. Add Output Link

In [9]:
WC_CRM_1_output = "C:/Users/aleksandar.dimitrov/Desktop/PD/Output/WC_CRM_1_October_2023.xlsx"

In [10]:
WC_CRM_7_output = "C:/Users/aleksandar.dimitrov/Desktop/PD/Output/WC_CRM_7_October_2023.xlsx"

In [11]:
A1_CRM_output = "C:/Users/aleksandar.dimitrov/Desktop/PD/Output/A1_CRM_October_2023.xlsx"

In [12]:
Raffle_laptops_output = "C:/Users/aleksandar.dimitrov/Desktop/PD/Output/Raffle_Philips_10.2023.xlsx"

## 4. Wait for the program

### Importing Libraries

In [13]:
import pandas as pd
import pyodbc
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

import win32com.client
from datetime import datetime

### Templates Loading

In [14]:
WC_CRM_1_template = load_workbook(WC_CRM_1__path)
worksheet_1 = WC_CRM_1_template.active

In [15]:
WC_CRM_7_template = load_workbook(WC_CRM_7__path)
worksheet_2 = WC_CRM_7_template.active

In [16]:
A1_CRM_template = load_workbook(A1_CRM__path)
worksheet_3 = A1_CRM_template.active

In [17]:
Raffle_laptops_template = load_workbook(Raffle_laptops__path)
worksheet_4 = Raffle_laptops_template.active

### SQL Querries Loading

In [18]:
server = "scorpio"
database = "BIsmartWCBG"
conn = pyodbc.connect(f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes")

In [19]:
sql_query1 = f"""

DECLARE @StartDate DATE = '{start_date_1}';
DECLARE @EndDate DATE = '{end_date_1}';
SELECT 
     ContractNumber
       , MIN(CONVERT(Date,fct.CDate)) as MinDate
                 from dwh.FactCardTransactions fct
                 JOIN dwh.DimOffers do ON do.OfferSK = fct.OfferSK
                 JOIN DWh.DimProduct p ON p.ProductSK = do.ProductSK
                 JOIN dwh.DimTransactionChannels tc ON tc.TransactionChannelSK = fct.TransactionChannelSK
                 WHERE tc.Name NOT IN ('ANNUAL_TAX', 'BALANCE', 'PIN_CHANGE', 'FEE')
                 AND ResponseCodeSK = 47
                 AND AmountTransactionBilling < 0 
                 AND CONVERT(DATE, fct.CDate, 4) >= @StartDate
                 AND CONVERT(DATE, fct.CDate, 4) <= @EndDate
Group by ContractNumber 
"""

In [20]:
sql_query2 = f"""

DECLARE @StartDate DATE = '{start_date_2}';
DECLARE @EndDate DATE = '{end_date_2}';
 
 SELECT
ContractNumber,
--, OfferSK,
 CDate AS Date
,CASE 
   WHEN Name = 'A1 - 8.5%'
      THEN 'A1'
   WHEN Name = 'AXI 2-500'
      THEN 'AXI 2'
   WHEN Name = 'Visa Free ATM World'
      THEN 'WhiteCard'
   WHEN Name = 'Visa Бяла карта'
      THEN 'WhiteCard'
   WHEN Name = 'Бяла Карта'
      THEN 'WhiteCard'
   WHEN Name = 'ИАМ'
      THEN 'WhiteCard'
   WHEN Name = '180 дни Visa Бяла Карта'
      THEN 'WhiteCard'
   WHEN Name = 'Аванс БГ Пощи'
      THEN 'WhiteCard'
   WHEN Name = 'Бяла Карта – само лихва'
      THEN 'WhiteCard only interest'
   WHEN Name = 'Visa Бяла Карта - SC'
      THEN 'WhiteCard'
   WHEN Name = 'Бяла Карта Gold - 5%'
      THEN 'WhiteCard-Gold'
   WHEN Name = 'Бяла Карта Gold - 10%'
      THEN 'WhiteCard-Gold'
   WHEN Name = 'Бяла Карта Gold-MFG-5%'
      THEN 'WhiteCard-Gold'
   WHEN Name = 'Кредит от БК 3'
      THEN 'Axi Credit 3'
   WHEN Name = 'Кредит от БК 4 '
      THEN 'Axi Credit 4'
   WHEN Name = 'Кредит от БК 4 - paper'
      THEN 'Axi Credit 4'
   WHEN Name = 'Бяла Карта 2'
      THEN 'WhiteCard'
ELSE Name
END AS Product

,CASE 
   WHEN TransactionChanel = 'POSCASH'
      THEN 'POS'
   WHEN TransactionChanel = 'MANUAL'
      THEN 'POS'
   WHEN TransactionChanel = 'ECOMM'
      THEN 'ONLINE'
   ELSE TransactionChanel
   END AS TransactionChanel

,CASE
When Name in ('AXI 2', 'AXI 2-500', 'Бяла Карта Gold - 5%', 'Бяла Карта Gold - 10%' ) and TransactionChanel = 'ATM'
then SUM(ATMAmount) 
Else  SUM(AmountTransactionBilling * -1) 
END AS AmountReal

, SUM(AmountTransactionBilling * -1) AS Amount
, COUNT(AmountTransactionBilling) AS CountTransaction
, SUM(ATMAmount) AS AxiATM
 FROM(
SELECT --fct.OfferSK
 ContractNumber
, fct.CDate
, p.Name
, AmountTransactionBilling
, TransactionDate
, tc.Name AS TransactionChanel,
 ((-AmountTransactionBilling - 1.5)/1.03) AS ATMAmount
   from dwh.FactCardTransactions fct
   JOIN dwh.DimOffers do ON do.OfferSK = fct.OfferSK
   JOIN DWh.DimProduct p ON p.ProductSK = do.ProductSK
   JOIN dwh.DimTransactionChannels tc ON tc.TransactionChannelSK = fct.TransactionChannelSK
   WHERE tc.Name NOT IN ('ANNUAL_TAX', 'BALANCE', 'PIN_CHANGE', 'FEE')
   AND ResponseCodeSK = 47
   AND AmountTransactionBilling < 0 
   AND fct.CDate >= @StartDate 
       AND fct.CDate <= @EndDate
   ) AS Result
     GROUP BY Name, TransactionChanel, CDate, ContractNumber -- OfferSK
   ORDER BY Date, Name
"""

In [21]:
sql_query3 = f"""

SELECT DISTINCT 
    cards.EasyClientNumber,
    prod.Name,
    MIN(cards.ActivationDate) AS FirstActivationDate,
    offer.DateGenerateContract AS ContractDate
FROM dwh.DimCards AS cards
LEFT JOIN dwh.DimCardProcessors AS cp ON cp.CardProcessorID = cards.CardProcessorsSK
LEFT JOIN dwh.DimOffers AS offer ON offer.OfferSK = cards.OfferSK
LEFT JOIN dwh.DimProduct AS prod ON prod.ProductSK = offer.ProductSK
WHERE cards.CardProcessorsSK = 2
GROUP BY cards.EasyClientNumber, prod.Name, offer.DateGenerateContract
ORDER BY cards.EasyClientNumber;

"""

In [22]:
data_sql_1 = pd.read_sql_query(sql_query1, conn)
data_sql_1 = data_sql_1.rename(columns={"ContractNumber": "EasyClientNumber"})
data_sql_1

C:\Users\aleksandar.dimitrov\AppData\Local\Temp\ipykernel_10720\3279117453.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_sql_1 = pd.read_sql_query(sql_query1, conn)


,EasyClientNumber,MinDate
0,400146,2023-10-03
1,402136,2023-10-02
2,408664,2023-10-03
3,418669,2023-10-01
4,420173,2023-10-07
...,...,...
48241,11070593,2023-10-13
48242,11070831,2023-10-11
48243,11071205,2023-10-13
48244,11071648,2023-10-17


In [23]:
data_sql_2 = pd.read_sql_query(sql_query2, conn)
data_sql_2 = data_sql_2.rename(columns={"ContractNumber": "ClientId"})
data_sql_2

C:\Users\aleksandar.dimitrov\AppData\Local\Temp\ipykernel_10720\1672454526.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_sql_2 = pd.read_sql_query(sql_query2, conn)


,ClientId,Date,Product,TransactionChanel,AmountReal,Amount,CountTransaction,AxiATM
0,520544,2023-10-10 00:00:12.367,WhiteCard,ATM,20.00,20.00,1,17.961165
1,11026066,2023-10-10 00:00:34.023,WhiteCard,ONLINE,50.00,50.00,1,47.087379
2,11052612,2023-10-10 00:00:48.320,Бяла Карта 3,ONLINE,10.00,10.00,1,8.252427
3,11054846,2023-10-10 00:01:09.833,Бяла Карта 3,ONLINE,20.00,20.00,1,17.961165
4,11058062,2023-10-10 00:01:38.507,Бяла Карта 3,ONLINE,40.00,40.00,1,37.378641
...,...,...,...,...,...,...,...,...
223492,829856,2023-10-31 23:58:48.987,Бяла Карта 3 - migrated,POS,99.86,99.86,1,95.495146
223493,11054846,2023-10-31 23:59:17.320,Бяла Карта 3,ONLINE,13.60,13.60,1,11.747573
223494,11004923,2023-10-31 23:59:38.797,WhiteCard-Gold,POS,12.64,12.64,1,10.815534
223495,814223,2023-10-31 23:59:38.977,Бяла Карта 3 - migrated,ONLINE,20.00,20.00,1,17.961165


In [24]:
data_sql_3 = pd.read_sql_query(sql_query3, conn)
data_sql_3 = data_sql_3.rename(columns={"EasyClientNumber": "ClientId"})
data_sql_3.drop(['Name', 'FirstActivationDate'], axis=1, inplace=True)
data_sql_3

C:\Users\aleksandar.dimitrov\AppData\Local\Temp\ipykernel_10720\2512416352.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_sql_3 = pd.read_sql_query(sql_query3, conn)


,ClientId,ContractDate
0,400007,2015-01-27 11:47:43.873
1,400025,2013-05-21 14:25:31.847
2,400026,2013-05-21 12:07:48.227
3,400028,2013-05-21 14:04:32.200
4,400033,2013-05-21 13:45:25.343
...,...,...
236604,11076030,NaT
236605,11076031,NaT
236606,11076033,NaT
236607,11076038,NaT


### Merged data_sql_2 and data_sql_3

In [25]:
merged_sql = data_sql_2.merge(data_sql_3, on="ClientId", how="left")
merged_sql

,ClientId,Date,Product,TransactionChanel,AmountReal,Amount,CountTransaction,AxiATM,ContractDate
0,520544,2023-10-10 00:00:12.367,WhiteCard,ATM,20.00,20.00,1,17.961165,2017-10-30 14:07:41.407
1,11026066,2023-10-10 00:00:34.023,WhiteCard,ONLINE,50.00,50.00,1,47.087379,2022-10-05 09:28:01.190
2,11052612,2023-10-10 00:00:48.320,Бяла Карта 3,ONLINE,10.00,10.00,1,8.252427,2023-05-26 10:53:59.383
3,11054846,2023-10-10 00:01:09.833,Бяла Карта 3,ONLINE,20.00,20.00,1,17.961165,2023-06-06 09:53:04.183
4,11058062,2023-10-10 00:01:38.507,Бяла Карта 3,ONLINE,40.00,40.00,1,37.378641,2023-06-30 09:26:59.600
...,...,...,...,...,...,...,...,...,...
223492,829856,2023-10-31 23:58:48.987,Бяла Карта 3 - migrated,POS,99.86,99.86,1,95.495146,2020-05-14 12:23:50.343
223493,11054846,2023-10-31 23:59:17.320,Бяла Карта 3,ONLINE,13.60,13.60,1,11.747573,2023-06-06 09:53:04.183
223494,11004923,2023-10-31 23:59:38.797,WhiteCard-Gold,POS,12.64,12.64,1,10.815534,2022-04-21 09:57:18.607
223495,814223,2023-10-31 23:59:38.977,Бяла Карта 3 - migrated,ONLINE,20.00,20.00,1,17.961165,2019-12-11 11:31:39.240


### Data Generation

In [26]:
start_date = datetime.strptime(start_date_1, '%Y-%m-%d').strftime('%d.%m')
end_date = datetime.strptime(end_date_1, '%Y-%m-%d').strftime('%d.%m')
period = start_date + "-" + end_date

In [27]:
desired_columns = ["EasyClientNumber"]

##### 1. WC_CRM_1_ Generation

In [28]:
WC_CRM_1_ = pd.read_excel(WC_CRM_1__path)
WC_CRM_1_ = WC_CRM_1_[desired_columns]
WC_CRM_1_ = WC_CRM_1_.merge(data_sql_1, on="EasyClientNumber", how="left")
WC_CRM_1_["MinDate"] = pd.to_datetime(WC_CRM_1_["MinDate"]).dt.strftime('%d.%m.%Y')
WC_CRM_1_["MinDate"].fillna("#N/A",inplace=True) 
WC_CRM_1_.rename(columns={'MinDate': f'Дата на първата трансакция в периода: {period} (ако има налична трансакция в този период)'}, inplace=True)
WC_CRM_1_

,EasyClientNumber,Дата на първата трансакция в периода: 01.10-31.10 (ако има налична трансакция в този период)
0,851702,#N/A
1,849000,#N/A
2,852408,#N/A
3,841585,#N/A
4,547043,26.10.2023
...,...,...
1700,11063068,#N/A
1701,11063306,#N/A
1702,11063187,#N/A
1703,11041215,#N/A


##### 2. WC_CRM_7_ Generation

In [29]:
WC_CRM_7_ = pd.read_excel(WC_CRM_7__path)
WC_CRM_7_ = WC_CRM_7_[desired_columns]
WC_CRM_7_ = WC_CRM_7_.merge(data_sql_1, on="EasyClientNumber", how="left")
WC_CRM_7_["MinDate"] = pd.to_datetime(WC_CRM_7_["MinDate"]).dt.strftime('%d.%m.%Y')
WC_CRM_7_["MinDate"].fillna("#N/A",inplace=True) 
WC_CRM_7_.rename(columns={'MinDate': f'Дата на първата трансакция в периода: {period} (ако има налична трансакция в този период)'}, inplace=True)
WC_CRM_7_

,EasyClientNumber,Дата на първата трансакция в периода: 01.10-31.10 (ако има налична трансакция в този период)
0,11047398,#N/A
1,11048027,#N/A
2,11045178,28.10.2023
3,11044878,#N/A
4,11048119,#N/A
...,...,...
146,929018,#N/A
147,920552,#N/A
148,898363,#N/A
149,869615,#N/A


##### 3. A1_CRM_ Generation

In [30]:
A1_CRM_ = pd.read_excel(A1_CRM__path)
A1_CRM_ = A1_CRM_[desired_columns]
A1_CRM_ = A1_CRM_.merge(data_sql_1, on="EasyClientNumber", how="left")
A1_CRM_["MinDate"] = pd.to_datetime(A1_CRM_["MinDate"]).dt.strftime('%d.%m.%Y')
A1_CRM_["MinDate"].fillna("#N/A",inplace=True) 
A1_CRM_.rename(columns={'MinDate': f'Дата на първата трансакция в периода: {period} (ако има налична трансакция в този период)'}, inplace=True)
A1_CRM_

,EasyClientNumber,Дата на първата трансакция в периода: 01.10-31.10 (ако има налична трансакция в този период)
0,572683,#N/A
1,805659,#N/A
2,581624,#N/A
3,581161,#N/A
4,827076,#N/A
...,...,...
576,11062777,05.10.2023
577,11063320,#N/A
578,11063008,#N/A
579,11063389,#N/A


##### 4. Raffle_laptops_ Generation

In [31]:
merged_sql["Date"] = pd.to_datetime(merged_sql["Date"]).dt.strftime('%d.%m.%Y')
merged_sql["ContractDate"] = pd.to_datetime(merged_sql["ContractDate"]).dt.strftime('%d.%m.%Y')

desired_coll = ["ClientId",
                "AmountReal",
                "TransactionChanel",
                "Date",
                "ContractDate"]

Raffle_laptops_ = merged_sql[desired_coll]
Raffle_laptops_ = Raffle_laptops_.sort_values(by="ClientId")
Raffle_laptops_ = Raffle_laptops_.reset_index(drop=True)
Raffle_laptops_

,ClientId,AmountReal,TransactionChanel,Date,ContractDate
0,400025,1.60,POS,24.10.2023,21.05.2013
1,400025,1.60,POS,20.10.2023,21.05.2013
2,400025,1.60,POS,25.10.2023,21.05.2013
3,400025,1.60,POS,21.10.2023,21.05.2013
4,400028,200.00,ATM,19.10.2023,21.05.2013
...,...,...,...,...,...
223492,11073484,50.00,ATM,31.10.2023,27.10.2023
223493,11073484,63.82,POS,31.10.2023,27.10.2023
223494,11073503,50.00,ATM,31.10.2023,27.10.2023
223495,11073503,100.00,ATM,31.10.2023,27.10.2023


### Data Export

##### 1. WC_CRM_1_ Export

In [32]:
data_rows = list(dataframe_to_rows(WC_CRM_1_, index=False, header=False))

In [33]:
start_row = 2
start_column = 1

In [34]:
for row in data_rows:
    for value in row:
        worksheet_1.cell(row=start_row, column=start_column, value=value)
        start_column += 1
    start_column = 1
    start_row += 1

In [35]:
WC_CRM_1_template.save(WC_CRM_1_output)

##### 2. WC_CRM_7_ Export

In [36]:
data_rows = list(dataframe_to_rows(WC_CRM_7_, index=False, header=False))

In [37]:
start_row = 2
start_column = 1

In [38]:
for row in data_rows:
    for value in row:
        worksheet_2.cell(row=start_row, column=start_column, value=value)
        start_column += 1
    start_column = 1
    start_row += 1

In [39]:
WC_CRM_7_template.save(WC_CRM_7_output)

##### 3. A1_CRM_ Export

In [40]:
data_rows = list(dataframe_to_rows(A1_CRM_, index=False, header=False))

In [41]:
start_row = 2
start_column = 1

In [42]:
for row in data_rows:
    for value in row:
        worksheet_3.cell(row=start_row, column=start_column, value=value)
        start_column += 1
    start_column = 1
    start_row += 1

In [43]:
A1_CRM_template.save(A1_CRM_output)

##### 4. Raffle_laptops_ Export

In [44]:
data_rows = list(dataframe_to_rows(Raffle_laptops_, index=False, header=False))

In [45]:
start_row = 2
start_column = 1

In [46]:
for row in data_rows:
    for value in row:
        worksheet_4.cell(row=start_row, column=start_column, value=value)
        start_column += 1
    start_column = 1
    start_row += 1

In [47]:
Raffle_laptops_template.save(Raffle_laptops_output)

### Email Anouncing

In [48]:
recipient_email = "alexi.zein@gmail.com"
subject = "Awards"
message = """Dear colleagues,

This email is automatically generated and contains information about added data.

Date and time of generation: {}

The Awards data has been added successfully and can be found in the shared folder on PD.

""".format(datetime.now().strftime("%d %B %Y, %H:%M ч."))

In [49]:
outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")

In [50]:
mail = outlook.CreateItem(0)
mail.Subject = subject
mail.Body = message
mail.To = recipient_email

In [51]:
mail.Send()

In [52]:
print("The notification email has been successfully sent.")

The notification email has been successfully sent.


In [53]:
end_time = time.time()
execution_time_seconds = end_time - start_time
execution_time_minutes = execution_time_seconds / 60
print(f"The script executed for {execution_time_minutes:.2f} minutes.")

The script executed for 0.54 minutes.
